# **Evaluating the model**

## **Import**

In [ ]:
import sys
import tensorflow as tf

# Google Colab specific imports
from google.colab import drive
from google.colab.patches import cv2_imshow

# Package for DenseNet
from tensorflow.keras.applications import DenseNet201
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dropout
from tensorflow.keras.regularizers import l2
from sklearn.utils.class_weight import compute_class_weight
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, EarlyStopping

import matplotlib.pyplot as plt

In [ ]:
# Mount Google Drive
drive.mount('/content/drive')

In [ ]:
# Global variables
train_generator = None
val_generator = None
model = None
base_model = None
class_names = None

# File paths
FULL_DATA_PATH = path_to_your_dataset
model_path = path_to_model

# **load dataset**

In [ ]:
def data_preprocess():
    global train_generator, val_generator, train_dataset, val_dataset, class_names

    dataset_path = FULL_DATA_PATH
    data_augmentation = tf.keras.Sequential([
        tf.keras.layers.RandomFlip("horizontal_and_vertical"),
        tf.keras.layers.RandomRotation(0.3),  # Increased rotation randomness
        tf.keras.layers.RandomZoom(0.3),      # Increased zoom randomness
        tf.keras.layers.RandomContrast(0.2),  # Add random contrast adjustments
        tf.keras.layers.RandomBrightness(0.2),  # Add random brightness adjustments
        tf.keras.layers.Resizing(224, 224),    # Resize back to target size
    ])

    # Load training dataset
    train_dataset = tf.keras.utils.image_dataset_from_directory(
        dataset_path,
        validation_split=0.2,  # Reserve 20% for validation
        subset="training",
        seed=23,  # Seed for reproducibility
        image_size=(224, 224),
        batch_size=32
    )

    # Extract and store class names
    class_names = train_dataset.class_names

    # Apply data augmentation and normalization to training dataset
    normalization_layer = tf.keras.layers.Rescaling(1./255)
    train_generator = train_dataset.map(
        lambda x, y: (normalization_layer(data_augmentation(x)), y)
    )

    # Load validation dataset
    val_dataset = tf.keras.utils.image_dataset_from_directory(
        dataset_path,
        validation_split=0.2,  # Reserve 20% for validation
        subset="validation",
        seed=23,  # Seed for reproducibility
        image_size=(224, 224),
        batch_size=32
    )

    # Normalize validation dataset
    val_generator = val_dataset.map(
        lambda x, y: (normalization_layer(x), y)
    )

    # Prefetch datasets for improved performance
    AUTOTUNE = tf.data.AUTOTUNE
    train_generator = train_generator.prefetch(buffer_size=AUTOTUNE)
    val_generator = val_generator.prefetch(buffer_size=AUTOTUNE)

In [ ]:
data_preprocess()

# **Load Model**

In [ ]:
from tensorflow.keras.models import load_model

# Load the saved model
model = load_model(model_path)

# **Evaluation**

## **Summary of model**

In [ ]:
model.summary()

## **Metrics**

In [ ]:
import tensorflow as tf
import numpy as np
from sklearn.metrics import (
    confusion_matrix,
    classification_report,
    accuracy_score,
    precision_score,
    recall_score,
    f1_score
)

In [ ]:
def evaluate_model_and_compute_metrics(model, val_generator, class_names):
    y_true = []
    y_pred = []

    # Iterate through the validation dataset
    for images, labels in val_generator:
        predictions = model.predict(images)  # Get predictions
        y_true.extend(labels.numpy())  # Append true labels
        y_pred.extend(np.argmax(predictions, axis=1))  # Append predicted class indices

    # Convert lists to numpy arrays
    y_true = np.array(y_true)
    y_pred = np.array(y_pred)

    # Compute metrics
    conf_matrix = confusion_matrix(y_true, y_pred)
    accuracy = accuracy_score(y_true, y_pred)
    precision_macro = precision_score(y_true, y_pred, average='macro')
    recall_macro = recall_score(y_true, y_pred, average='macro')
    f1_macro = f1_score(y_true, y_pred, average='macro')
    precision_weighted = precision_score(y_true, y_pred, average='weighted')
    recall_weighted = recall_score(y_true, y_pred, average='weighted')
    f1_weighted = f1_score(y_true, y_pred, average='weighted')
    class_report = classification_report(y_true, y_pred, target_names=class_names)

    return {
        "conf_matrix": conf_matrix,
        "accuracy": accuracy,
        "precision_macro": precision_macro,
        "recall_macro": recall_macro,
        "f1_macro": f1_macro,
        "precision_weighted": precision_weighted,
        "recall_weighted": recall_weighted,
        "f1_weighted": f1_weighted,
        "class_report": class_report
    }

# Evaluate the model
metrics = evaluate_model_and_compute_metrics(model, val_generator, class_names)

# Print results
print(f"Accuracy: {metrics['accuracy']:.4f}")
print(f"Macro Precision: {metrics['precision_macro']:.4f}")
print(f"Macro Recall: {metrics['recall_macro']:.4f}")
print(f"Macro F1-Score: {metrics['f1_macro']:.4f}")
print(f"Weighted Precision: {metrics['precision_weighted']:.4f}")
print(f"Weighted Recall: {metrics['recall_weighted']:.4f}")
print(f"Weighted F1-Score: {metrics['f1_weighted']:.4f}")
print("\nConfusion Matrix:\n", metrics['conf_matrix'])
print("\nClassification Report:\n", metrics['class_report'])


In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import matplotlib.gridspec as gridspec

In [ ]:
def plot_class_report_with_images_and_text_tighter_with_grid(class_report, class_names, val_generator):
    import matplotlib.pyplot as plt
    import pandas as pd
    import numpy as np

    # Parse the class_report string into a DataFrame
    lines = class_report.split("\n")
    report_data = []
    for line in lines[2 : len(class_names) + 2]:  # Skip headers and totals
        row = line.split()
        if len(row) >= 5:  # Ensure valid row structure
            class_name = " ".join(row[:-4])  # Handles multi-word class names
            precision, recall, f1_score, support = map(float, row[-4:])
            report_data.append([class_name, precision, recall, f1_score, support])
    class_report_df = pd.DataFrame(
        report_data, columns=["Class Name", "Precision", "Recall", "F1-Score", "Support"]
    )

    # Extract one example image per class from the validation generator
    example_images = {}
    for images, labels in val_generator.unbatch().take(1500):  # Unbatch and take enough examples
        label = labels.numpy()
        if label not in example_images:
            example_images[label] = images.numpy()
        if len(example_images) == len(class_names):  # Stop if we have one image per class
            break

    # Set up the plot for 5 results per row
    n_cols = 5
    n_rows = int(np.ceil(len(class_names) / n_cols))
    fig, axes = plt.subplots(
        n_rows, n_cols * 2, figsize=(15, 20)
    )  # Double columns for image + text, larger figure size
    spec = gridspec.GridSpec(n_rows, n_cols * 2, figure=fig, width_ratios=[1, 3] * n_cols)

    # Iterate through rows and columns
    for idx, row in class_report_df.iterrows():
        row_idx = idx // n_cols
        col_idx = (idx % n_cols) * 2  # Double column index for image + text

        # Show the image on the left
        ax_image = axes[row_idx, col_idx] if n_rows > 1 else axes[col_idx]
        if idx in example_images:
            ax_image.imshow(example_images[idx])
        ax_image.set_xticks([])  # Remove x ticks
        ax_image.set_yticks([])  # Remove y ticks

        # Show the metrics as text on the right
        ax_text = axes[row_idx, col_idx + 1] if n_rows > 1 else axes[col_idx + 1]
        metrics_text = (
            f"{row['Class Name']}\n"
            f"Precision: {row['Precision']:.2f}\n"
            f"Recall:    {row['Recall']:.2f}\n"
            f"F1-Score: {row['F1-Score']:.2f}\n"
            f"Support:  {int(row['Support'])}"
        )
        ax_text.text(0, 0.5, metrics_text, fontsize=8, va="center", ha="left")
        ax_text.axis("on")  # Show grid around text
        ax_text.set_xticks([])  # Remove x ticks
        ax_text.set_yticks([])  # Remove y ticks


    # Hide any unused axes
    for ax in axes.flatten():
        if not ax.has_data():
            ax.axis("off")

    plt.tight_layout()
    plt.subplots_adjust(top=0.92)
    plt.show()


plot_class_report_with_images_and_text_tighter_with_grid(metrics["class_report"], class_names, val_generator)


## **Confusion Matrix**

In [ ]:
def plot_confusion_matrix_with_color_left_bar(conf_matrix, class_names):
    import matplotlib.pyplot as plt
    import numpy as np

    # Set up the figure and axes with larger size
    fig, ax = plt.subplots(figsize=(20, 20))  # Increased figure size

    # Add color bar on the left
    cax = ax.matshow(conf_matrix, cmap='Blues')
    cbar = fig.colorbar(cax, ax=ax, fraction=0.06, pad=0.04, location='left')

    # Display axis numbers for classes
    num_classes = len(class_names)
    ax.set_xticks(range(1, num_classes + 1))
    ax.set_yticks(range(1, num_classes + 1))
    ax.set_xticklabels(range(1, num_classes + 1), rotation=90)  # Use numbers for axis
    ax.set_yticklabels(range(1, num_classes + 1))  # Use numbers for axis
    ax.xaxis.set_label_position("bottom")
    ax.xaxis.tick_bottom()

    # Add class labels on the sides with corresponding numbers
    side_labels = [f"{i + 1}: {label}" for i, label in enumerate(class_names)]
    for idx, label in enumerate(side_labels):
        ax.text(num_classes + 1.5, idx, label, va='center', ha='left', fontsize=8)

    # Set axis labels
    ax.set_xlabel("Predicted Labels")
    ax.set_ylabel("True Labels")
    plt.title("Confusion Matrix (Color Intensity)")

    # Adjust layout for the color bar and matrix
    plt.subplots_adjust(left=0.1)  # Reserve space for the color bar
    plt.show()

# Example usage
plot_confusion_matrix_with_color_left_bar(metrics['conf_matrix'], class_names)
